In [167]:
import os
import glob
import pdfplumber
import re
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# check working directory

In [29]:
root = os.getcwd()
ppt_folder = "/ppt"
to_pdf_folder = "/to_pdf"
root+ppt_folder

'/home/dquintubu/Documents/boligrafica-dquintubu/vacunacion-municipal/ppt'

# check existing files

In [16]:
list_ppts = os.listdir(root+ppt_folder)
list_ppts

['20nov2021 Cobertura de Vacunación, por Departamento v.10.pptx',
 '30oct2021 Cobertura de Vacunación, por Departamento (2da Dosis) v.10.pptx',
 '13nov2021 Cobertura de Vacunación, por Departamento (2da Dosis) v.10.pptx',
 '23oct2021 Cobertura de Vacunación, por Departamento v.10.pptx',
 '13nov2021 Cobertura de Vacunación, por Departamento v.10.pptx',
 '23oct2021 Cobertura de Vacunación, por Departamento (2da Dosis) v.10.pptx',
 '30oct2021 Cobertura de Vacunación, por Departamento v.10.pptx',
 '20nov2021 Cobertura de Vacunación, por Departamento (2da Dosis) v.10.pptx']

In [17]:
dir = root + to_pdf_folder
list_pdfs = glob.glob(os.path.join(dir,"*.pdf*"))
list_pdfs

['/home/dquintubu/Documents/boligrafica-dquintubu/vacunacion-municipal/to_pdf/23oct2021 Cobertura de Vacunación, por Departamento v.10.pdf',
 '/home/dquintubu/Documents/boligrafica-dquintubu/vacunacion-municipal/to_pdf/13nov2021 Cobertura de Vacunación, por Departamento (2da Dosis) v.10.pdf',
 '/home/dquintubu/Documents/boligrafica-dquintubu/vacunacion-municipal/to_pdf/20nov2021 Cobertura de Vacunación, por Departamento (2da Dosis) v.10.pdf',
 '/home/dquintubu/Documents/boligrafica-dquintubu/vacunacion-municipal/to_pdf/30oct2021 Cobertura de Vacunación, por Departamento v.10.pdf',
 '/home/dquintubu/Documents/boligrafica-dquintubu/vacunacion-municipal/to_pdf/23oct2021 Cobertura de Vacunación, por Departamento (2da Dosis) v.10.pdf',
 '/home/dquintubu/Documents/boligrafica-dquintubu/vacunacion-municipal/to_pdf/13nov2021 Cobertura de Vacunación, por Departamento v.10.pdf',
 '/home/dquintubu/Documents/boligrafica-dquintubu/vacunacion-municipal/to_pdf/30oct2021 Cobertura de Vacunación, por D

# convert ppt to pdf

In [18]:
if len(list_ppts) != len(list_pdfs):
    for ppt in list_ppts:
        os.system("unoconv -f pdf -o {} ppt/".format(root + to_pdf_folder + "/" + "\"" + ppt + "\"") + "\"" + ppt + "\"")
        print("converted " + ppt + " to pdf")
else:
    print("all of them already converted!")

all of them already converted!


# edu name correction

In [32]:
edu_merger = pd.read_csv("https://gist.githubusercontent.com/pr0nstar/df9829a59214fea0c7f9b70913c0af2c/raw/d872c1c53fc6e1ae7ff64c6fa06756b6ff2ae06a/vac.muni.csv", index_col=0)
edu_merger

,c_ut,municipio,departamento
0,10101,Sucre,Chuquisaca
1,10102,Yotala,Chuquisaca
2,10103,Poroma,Chuquisaca
3,10201,Azurduy,Chuquisaca
4,10202,Tarvita,Chuquisaca
...,...,...,...
334,90401,Nacebe (Santa Rosa Del Abuna),Pando
335,90402,Ingavi (Humaita),Pando
336,90501,Nuevo Manoa (Nueva Esperanza),Pando
337,90502,Villa Nueva (Loma Alta),Pando


# fuckaroo plumbing

In [261]:
for j in range(len(list_pdfs)):
    try:
        file = list_pdfs[j]
        print("FILE: ", file)
        pdf = pdfplumber.open(file)

        dfz = pd.DataFrame([])
        for k in range(1,len(pdf.pages)):
            page_number = k
            print("\nPAGE #: " , page_number+1)
            page = pdf.pages[page_number]

            table = page.extract_tables(table_settings={
                "vertical_strategy": "lines", 
                "horizontal_strategy": "lines",
            }
            )
            # print(table)
            table_number = len(table)
            print("\n# TABLES: ", table_number) 

            df = pd.DataFrame([])
            for i in range(table_number):
                df_raw = pd.DataFrame(table[i])
                # display(df_raw)

                if len(df_raw.columns)>2:
                    print("whoops, too fat, lets separate these")
                    df_sep1 = df_raw.iloc[:,[0,1]].set_index(0)
                    df_sep1 = df_sep1.rename({1:"cobertura"}, axis=1)
                    df_sep2 = df_raw.iloc[:,[2,3]].set_index(2)
                    df_sep2 = df_sep2.rename({3:"cobertura"}, axis=1)
                    df_skinny = pd.concat([df_sep1, df_sep2], axis=0, ignore_index=False)
                else:
                    df_skinny = df_raw.set_index(0)
                    df_skinny = df_skinny.rename({1:"cobertura"}, axis=1)

                df = pd.concat([df, df_skinny], axis=0)

            # display(df)
            df = df[df.index.notnull()]
            df = df[df.index != '']

            df.cobertura = df.cobertura.str.replace("%","")
            df.cobertura = df.cobertura.astype(float)
            # display(df)
            # df.sort_values("cobertura", ascending=False).plot()

            dfz = pd.concat([dfz, df], axis=0)
        display(dfz.sort_values("cobertura", ascending=False))
    except:
        print("FUCK! DID NOT WORK. be sad \n \n \n \n")

FILE:  /home/dquintubu/Documents/boligrafica-dquintubu/vacunacion-municipal/to_pdf/23oct2021 Cobertura de Vacunación, por Departamento v.10.pdf

PAGE #:  2

# TABLES:  4
whoops, too fat, lets separate these

PAGE #:  3

# TABLES:  5
whoops, too fat, lets separate these

PAGE #:  4

# TABLES:  6

PAGE #:  5

# TABLES:  5
whoops, too fat, lets separate these

PAGE #:  6

# TABLES:  5
whoops, too fat, lets separate these

PAGE #:  7

# TABLES:  3

PAGE #:  8

# TABLES:  4

PAGE #:  9

# TABLES:  5

PAGE #:  10

# TABLES:  5


,cobertura
Santivañes,130.0
La Paz,102.0
Huachacalla,98.0
Cuchumuela (V. G. Villarroel),98.0
Cochabamba,96.0
...,...
Escara,9.0
Cruz De Machacamarca,6.0
Coberturas Baja,NaN
Cobertura Muy Baja,NaN


FILE:  /home/dquintubu/Documents/boligrafica-dquintubu/vacunacion-municipal/to_pdf/13nov2021 Cobertura de Vacunación, por Departamento (2da Dosis) v.10.pdf

PAGE #:  2

# TABLES:  1
FUCK! DID NOT WORK. be sad 
 
 
 

FILE:  /home/dquintubu/Documents/boligrafica-dquintubu/vacunacion-municipal/to_pdf/20nov2021 Cobertura de Vacunación, por Departamento (2da Dosis) v.10.pdf

PAGE #:  2

# TABLES:  1
FUCK! DID NOT WORK. be sad 
 
 
 

FILE:  /home/dquintubu/Documents/boligrafica-dquintubu/vacunacion-municipal/to_pdf/30oct2021 Cobertura de Vacunación, por Departamento v.10.pdf

PAGE #:  2

# TABLES:  4
whoops, too fat, lets separate these

PAGE #:  3

# TABLES:  5
whoops, too fat, lets separate these

PAGE #:  4

# TABLES:  6

PAGE #:  5

# TABLES:  4
whoops, too fat, lets separate these

PAGE #:  6

# TABLES:  6

PAGE #:  7

# TABLES:  3

PAGE #:  8

# TABLES:  4

PAGE #:  9

# TABLES:  5

PAGE #:  10

# TABLES:  5


,cobertura
Santivañes,131.0
La Paz,102.0
Cuchumuela (V. G. Villarroel),100.0
Huachacalla,98.0
Cochabamba,97.0
...,...
Presto,NaN
Incahuasi,NaN
Culpina,NaN
El Villar,NaN


FILE:  /home/dquintubu/Documents/boligrafica-dquintubu/vacunacion-municipal/to_pdf/23oct2021 Cobertura de Vacunación, por Departamento (2da Dosis) v.10.pdf

PAGE #:  2

# TABLES:  1
FUCK! DID NOT WORK. be sad 
 
 
 

FILE:  /home/dquintubu/Documents/boligrafica-dquintubu/vacunacion-municipal/to_pdf/13nov2021 Cobertura de Vacunación, por Departamento v.10.pdf

PAGE #:  2

# TABLES:  5

PAGE #:  3

# TABLES:  5

PAGE #:  4

# TABLES:  4

PAGE #:  5

# TABLES:  4
whoops, too fat, lets separate these

PAGE #:  6

# TABLES:  4
whoops, too fat, lets separate these

PAGE #:  7

# TABLES:  3

PAGE #:  8

# TABLES:  4
whoops, too fat, lets separate these

PAGE #:  9

# TABLES:  6

PAGE #:  10

# TABLES:  5
whoops, too fat, lets separate these


,cobertura
Santivañes,133.0
La Paz,104.0
Cuchumuela (V. G. Villarroel),102.0
Huachacalla,99.0
Cochabamba,99.0
...,...
San Lucas,NaN
El Villar,NaN
Padilla,NaN
Coberturas Baja,NaN


FILE:  /home/dquintubu/Documents/boligrafica-dquintubu/vacunacion-municipal/to_pdf/30oct2021 Cobertura de Vacunación, por Departamento (2da Dosis) v.10.pdf

PAGE #:  2

# TABLES:  1
FUCK! DID NOT WORK. be sad 
 
 
 

FILE:  /home/dquintubu/Documents/boligrafica-dquintubu/vacunacion-municipal/to_pdf/20nov2021 Cobertura de Vacunación, por Departamento v.10.pdf

PAGE #:  2

# TABLES:  5

PAGE #:  3

# TABLES:  5

PAGE #:  4

# TABLES:  4

PAGE #:  5

# TABLES:  4
whoops, too fat, lets separate these

PAGE #:  6

# TABLES:  4
whoops, too fat, lets separate these

PAGE #:  7

# TABLES:  3

PAGE #:  8

# TABLES:  5
whoops, too fat, lets separate these

PAGE #:  9

# TABLES:  6

PAGE #:  10

# TABLES:  5
whoops, too fat, lets separate these


,cobertura
Santivañes,134.0
La Paz,104.0
Cuchumuela (V. G. Villarroel),103.0
Cochabamba,100.0
Huachacalla,99.0
...,...
San Lucas,NaN
El Villar,NaN
Padilla,NaN
Coberturas Baja,NaN


In [262]:
# def query_paginas(pdf, query):
    
#     for page in pdf.pages:
#         content = page.extract_text()
#         if content != None:
#             # if query in ' '.join(content.split()).replace(',','').lower():#normalizar texto
#             if query in content:
#                 return page
#         # print("page: ", page)
#         print(content)
#         print("querry found on " + str(page) + "of " + reporte)

# # def estandarizar_nombres(variaciones, nombres):
    
# #     if type(nombres) == list:
# #         return [variaciones[variacion] for variacion in variaciones.keys() for nombre in nombres if variacion in nombre]
# #     elif type(nombres) == str:
# #         return [variaciones[variacion] for variacion in variaciones.keys() if variacion in nombres][0]

# def extraer_tabla(page):
#     filas = page.extract_table()

        
# # ----------------------------
    

# for reporte in list_pdfs:
#     # Abrirlo
#     pdf = pdfplumber.open(reporte)
#     # De qué fecha es
#     # date = que_fecha(pdf)
#     date = str(list_pdfs[0])[0:9]
#     # En qué página está la tabla que me interesa
#     page = query_paginas(pdf, 'Cochabamba')
#     # Extraer la tabla
#     df = extraer_tabla(page)
#     # Guardarla
#     # consolidar(df)
#     # Extraer y consolidar coberturas por población y tipo de dosis
#     # cobertura(pdf, 'cobertura de población vacunada con 1ra dosis según', 'cobertura/por_edad_primera_dosis.csv')
#     # cobertura(pdf, 'cobertura de población vacunada con esquema completo', 'cobertura/por_edad_esquema_completo.csv')

In [7]:
print(df)

None


In [8]:
date = str(list_pdfs[0])[0:9]

In [9]:
if "yes" in "jkfldysa;jkd":
    print("words")